##  Process Phase(Data Cleaning)
The "Process" phase is where the data collected are being processed, this includes connecting the data sources, cleaning the data, filtering and choosing the data, calculating the confidence level of the data etc.

Deliverable: **documentation of any cleaning or manipulation of data**.

### 1 . Documentation of data cleaning and manipulation
I will be using **Pandas** for the data cleaning and manipulations. 
The actions that I carried out are as follows:

1. Since the data for each month are in a seperate file, I import each file individually.
2. Then I merge these files into one data frame which enable me to analyze them in term of the year 2023 December to 2024 November.
3. Identify the columns and type of each column of the data frame.
4. Check for null values.
    - Drop the null values (if any), and reconfirm that there are no longer null values
5. Delete the columns that are not necessary.
6. Check for duplicates.
    - Drop the duplicated values (if any), and reconfirm that there are no longer duplicated values
7. Modify the data type of certain column.
    - Convert _**started_at**_ column and _**ended_at**_ column from  "object" to "datetime64"
    - To **extract the hour, day, month and date** to form new columns based on them
    - To calculate the **ride duration**
8. Check the order of the day, month.
9. Create a new column _**ride_duration**_ that calculates the duration of each ride.
    - Express the ride duration in terms of only **minutes** , and not **minutes and seconds**
    - Check whether there are **negative values** for the ride duration which would be illogical
    - Remove the rows that have negative values
10. Export the file of the cleaned data.

### 1.1 Read each csv file individually

In [ ]:
%pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

In [ ]:
####Read the each file seprately and then merge them together
dec_23=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202312-divvy-tripdata.csv')
jan_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202401-divvy-tripdata.csv')
feb_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202402-divvy-tripdata.csv')
march_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202403-divvy-tripdata.csv')
april_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202404-divvy-tripdata.csv')
may_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202405-divvy-tripdata.csv')
june_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202406-divvy-tripdata.csv')
july_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202407-divvy-tripdata.csv')
aug_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202408-divvy-tripdata.csv')
sep_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202409-divvy-tripdata.csv')
oct_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202410-divvy-tripdata.csv')
nov_24=pd.read_csv('D:/Google-Data-Analytics-Case-Study-Python-main/case_study/202411-divvy-tripdata.csv')

#Merging all the data together
df=pd.concat([dec_23,jan_24,feb_24,march_24,april_24,may_24,june_24,july_24,aug_24,sep_24,oct_24,nov_24],ignore_index=True)


In [ ]:
### Identify the respective coloumns and their data types
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5906269 entries, 0 to 5906268
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 585.8+ MB


#### From this we can see that there are totally 12 coloumns and 4 coloums have a float64 has datatype nad remaining have object  has datatype

### 1.2 Checking for null values

In [ ]:
df.isnull().sum()

ride_id                     0
rideable_type               0
started_at                  0
ended_at                    0
start_station_name    1080470
start_station_id      1080470
end_station_name      1111801
end_station_id        1111801
start_lat                   0
start_lng                   0
end_lat                  7340
end_lng                  7340
member_casual               0
dtype: int64

We can see around 70K null values for some colums.These null values are the main obstacles while Data analysis.
Just remove those null values from our data

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

### 1.3 Deleting coloums that are unnecessary

In [ ]:
df = df.drop(['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id',
             'start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4244722 entries, 189 to 5906268
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ride_id        object
 1   rideable_type  object
 2   started_at     object
 3   ended_at       object
 4   member_casual  object
dtypes: object(5)
memory usage: 194.3+ MB


### 1.4 Checking for Duplicated values

In [ ]:
df.duplicated().sum()

np.int64(0)

We are all good to go since there are no duplicated values, if there are, we should drop them too.

### 1.5 Changing the Datatype

We will be converting the column type for two columns, __**started_at**__ column and __**ended_at**__ column from the data type **"object"** to the data type **"datetime64"**

There are 2 reasons for this:
- To **extract the hour, day, month and date** to form new columns based on them
- To calculate the **ride duration**

In [ ]:
# To chech the current data types of the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4244722 entries, 189 to 5906268
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ride_id        object
 1   rideable_type  object
 2   started_at     object
 3   ended_at       object
 4   member_casual  object
dtypes: object(5)
memory usage: 194.3+ MB


Now to calculate travel duration, we are converting the started_at and ended _at from  **Object** to **DateTime64**, a date dataframe type.

In [ ]:
df["started_at"] = df["started_at"].astype('datetime64[ns]')
df["ended_at"] = df["ended_at"].astype('datetime64[ns]')

# To check the updated data type
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 4244722 entries, 189 to 5906268
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ride_id        object        
 1   rideable_type  object        
 2   started_at     datetime64[ns]
 3   ended_at       datetime64[ns]
 4   member_casual  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 194.3+ MB


The `[ns]` in `datetime64[ns]` specifies the precision of the datetime values in a pandas DataFrame or Series. Here, `ns` stands for nanoseconds, which is the highest precision available for datetime objects in pandas.

When you see `datetime64[ns]`, it means that the datetime values are stored with nanosecond precision. This is useful for applications that require very high precision for time data, such as financial data analysis or scientific experiments


### 1.6 Create new columns that specify the hour, the day, the month, the year of each ride. 


In [ ]:
df['hour'] = df['started_at'].dt.hour # Toextract the hour
df['day'] = df['started_at'].dt.day_name() # Toextract the day of the week
df['date'] = df['started_at'].dt.day #To extract the date
df['month'] = df['started_at'].dt.month_name() # To extract the month
df['year'] = df['started_at'].dt.year #To extract the year


df.head()

,ride_id,rideable_type,started_at,ended_at,member_casual,hour,day,date,month,year
189,84BFC1F137684EAB,classic_bike,2023-12-02 23:12:51,2023-12-02 23:21:01,member,23,Saturday,2,December,2023
996,EEC92D30A70471E5,classic_bike,2023-12-14 13:43:14,2023-12-14 13:44:14,casual,13,Thursday,14,December,2023
1322,1C33464DEEB1F23C,electric_bike,2023-12-04 11:57:04,2023-12-04 12:13:59,casual,11,Monday,4,December,2023
1347,E0A61810C305E5EC,classic_bike,2023-12-04 09:34:22,2023-12-04 09:35:56,casual,9,Monday,4,December,2023
1348,0706CEB2E1924F3D,classic_bike,2023-12-04 09:36:27,2023-12-04 09:36:40,casual,9,Monday,4,December,2023


### 1.7 Check the order of the day, month

In [ ]:
df.groupby(['member_casual', 'day'])['ride_id'].count() # Check order for the day

member_casual  day      
casual         Friday       226254
               Monday       179862
               Saturday     320126
               Sunday       266857
               Thursday     185330
               Tuesday      162466
               Wednesday    189189
member         Friday       386510
               Monday       395210
               Saturday     339014
               Sunday       297296
               Thursday     422869
               Tuesday      420348
               Wednesday    453391
Name: ride_id, dtype: int64

### 1.8. Check the order of the day, month

In [ ]:
df.groupby(['member_casual', 'day'])['ride_id'].count() # Check order for the day

member_casual  day      
casual         Friday       226254
               Monday       179862
               Saturday     320126
               Sunday       266857
               Thursday     185330
               Tuesday      162466
               Wednesday    189189
member         Friday       386510
               Monday       395210
               Saturday     339014
               Sunday       297296
               Thursday     422869
               Tuesday      420348
               Wednesday    453391
Name: ride_id, dtype: int64

The days are not arranged in the order that we want, so we have to arrange them

In [ ]:
day_ordered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df['day'] = pd.Categorical(df['day'], categories=day_ordered, ordered=True)
df.groupby(['member_casual', 'day'])['ride_id'].count() 

C:\Users\mk\AppData\Local\Temp\ipykernel_10956\3152813243.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['member_casual', 'day'])['ride_id'].count()


member_casual  day      
casual         Monday       179862
               Tuesday      162466
               Wednesday    189189
               Thursday     185330
               Friday       226254
               Saturday     320126
               Sunday       266857
member         Monday       395210
               Tuesday      420348
               Wednesday    453391
               Thursday     422869
               Friday       386510
               Saturday     339014
               Sunday       297296
Name: ride_id, dtype: int64

The days are now ordered accordingly as we want, which is Monday - Friday.

In [ ]:
# Now we repeat for the month

df.groupby(['member_casual', 'month'])['ride_id'].count() # Check order for the month

member_casual  month    
casual         April         93944
               August       228518
               December      36686
               February      38170
               January       17713
               July         231970
               June         208397
               March         62821
               May          167552
               November      68816
               October      159354
               September    216143
member         April        203854
               August       312805
               December     130457
               February     146566
               January       96095
               July         308971
               June         285945
               March        167457
               May          274737
               November     177135
               October      289762
               September    320854
Name: ride_id, dtype: int64

The months are not arranged in the order that we want, so we have to arrange them.

In [ ]:
# Arrange the month

month_ordered = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October',
               'November', 'December']

df['month'] = pd.Categorical(df['month'], categories=month_ordered, ordered=True)
df.groupby(['member_casual', 'month'])['ride_id'].count() 

C:\Users\mk\AppData\Local\Temp\ipykernel_10956\195444796.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['member_casual', 'month'])['ride_id'].count()


member_casual  month    
casual         January       17713
               February      38170
               March         62821
               April         93944
               May          167552
               June         208397
               July         231970
               August       228518
               September    216143
               October      159354
               November      68816
               December      36686
member         January       96095
               February     146566
               March        167457
               April        203854
               May          274737
               June         285945
               July         308971
               August       312805
               September    320854
               October      289762
               November     177135
               December     130457
Name: ride_id, dtype: int64

Perfect! The months are now arranged as wanted, which is January - December.

### 1.9. Create new column that calculate the duration of each ride

In [ ]:
df['ride_duration'] = (df['ended_at'] - df['started_at'])/ pd.Timedelta(minutes=1) 
#Express the ride duration in terms of only minutes , and not minutes and seconds

df.head()

,ride_id,rideable_type,started_at,ended_at,member_casual,hour,day,date,month,year,ride_duration
189,84BFC1F137684EAB,classic_bike,2023-12-02 23:12:51,2023-12-02 23:21:01,member,23,Saturday,2,December,2023,8.166667
996,EEC92D30A70471E5,classic_bike,2023-12-14 13:43:14,2023-12-14 13:44:14,casual,13,Thursday,14,December,2023,1.000000
1322,1C33464DEEB1F23C,electric_bike,2023-12-04 11:57:04,2023-12-04 12:13:59,casual,11,Monday,4,December,2023,16.916667
1347,E0A61810C305E5EC,classic_bike,2023-12-04 09:34:22,2023-12-04 09:35:56,casual,9,Monday,4,December,2023,1.566667
1348,0706CEB2E1924F3D,classic_bike,2023-12-04 09:36:27,2023-12-04 09:36:40,casual,9,Monday,4,December,2023,0.216667


### 1.10. Verify if there are negative values for the duration of ride, which is not logical

In [ ]:
df[df['ride_duration'] < 0].count()

ride_id          44
rideable_type    44
started_at       44
ended_at         44
member_casual    44
hour             44
day              44
date             44
month            44
year             44
ride_duration    44
dtype: int64

Apparently there are 166 rides with negative durations which are not logical, so we have to delete them.

In [ ]:
df = df[df['ride_duration'] > 0] # Remove the rows
df[df['ride_duration'] < 0].count() # Verify

ride_id          0
rideable_type    0
started_at       0
ended_at         0
member_casual    0
hour             0
day              0
date             0
month            0
year             0
ride_duration    0
dtype: int64

We reconfirmed now that the rides with negative durations have been succesfully ommitted.

In [ ]:
df.sort_values(by = ['ride_duration'], ascending = True).head()

,ride_id,rideable_type,started_at,ended_at,member_casual,hour,day,date,month,year,ride_duration
5820442,CF11AD1AA5499FD6,classic_bike,2024-11-03 01:57:59.221,2024-11-03 01:02:33.679,member,1,Sunday,3,November,2024,-55.425700
5621638,F600DA96D6308ABF,electric_bike,2024-11-03 01:55:22.919,2024-11-03 01:00:36.080,casual,1,Sunday,3,November,2024,-54.780650
5632045,BB4F8E728A4714E3,classic_bike,2024-11-03 01:55:10.885,2024-11-03 01:02:13.083,casual,1,Sunday,3,November,2024,-52.963367
5848502,6E973218589BD118,classic_bike,2024-11-03 01:55:58.187,2024-11-03 01:03:34.688,member,1,Sunday,3,November,2024,-52.391650
5773991,C2D0EA867C94CB05,electric_bike,2024-11-03 01:57:10.166,2024-11-03 01:05:39.411,member,1,Sunday,3,November,2024,-51.512583


We can also verify by sorting the duration ascendingly, then identify whether the smallest value is positive.

We can see that the shortest duration which is 0.167 minute is positive, therefore all the negative values have been deleted succesfully.